setup

In [6]:
%pip install tensorflow tensorflow-gpu pandas matplotlib sklearn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ellbendl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:

import os
import pandas as pd
import tensorflow as tf
import numpy as np


In [2]:
os.path.join('jigsaw-toxic-comment-classification-challenge', 'train.csv', 'train.csv')

'jigsaw-toxic-comment-classification-challenge\\train.csv\\train.csv'

In [3]:
data = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge', 'train.csv', 'train.csv'))

In [4]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
data.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [6]:
data.iloc[43]['comment_text']

'FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!'

In [7]:
data[data.columns[2:]].iloc[43]

toxic            1
severe_toxic     0
obscene          1
threat           0
insult           1
identity_hate    0
Name: 43, dtype: int64

Data Preprocessing

In [8]:
!pip list

Package                       Version
----------------------------- --------------------
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
anaconda-client               1.9.0
anaconda-navigator            2.1.4
anaconda-project              0.10.2
analytics-python              1.4.0
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.6.6
astropy                       5.0.4
asttokens                     2.0.5
async-timeout                 4.0.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backoff                       1.10.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backpo

In [8]:
from tensorflow.keras.layers import TextVectorization

In [9]:
data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [10]:
x = data['comment_text']
y = data[data.columns[2:]].values

In [11]:
x

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [12]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [13]:
MAX_FEATURES = 200000

In [14]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')

In [15]:
vectorizer.adapt(x.values)

In [16]:
vectorizer('Yo Whats up')[:3]

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([2643,  650,   81], dtype=int64)>

In [17]:
vectorized_text = vectorizer(x.values)

In [18]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32141,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [19]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)


In [20]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [21]:
batch_y.shape

(16, 6)

In [22]:
int(len(dataset)*.7)

6981

In [23]:

train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))


In [24]:
train_generator = train.as_numpy_iterator()

In [25]:
train_generator.next()

(array([[ 1370,   941,   878, ...,     0,     0,     0],
        [  486,   279,   593, ...,     0,     0,     0],
        [   16,     6,   195, ...,     0,     0,     0],
        ...,
        [    7, 51087,     7, ...,     0,     0,     0],
        [   86,   636,    46, ...,     0,     0,     0],
        [  159,   862,    48, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

MOdel

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [27]:
modelnya = Sequential()

modelnya.add(Embedding(MAX_FEATURES+1, 32))

modelnya.add(Bidirectional(LSTM(32, activation='tanh')))

modelnya.add(Dense(128, activation='relu'))
modelnya.add(Dense(256, activation='relu'))

modelnya.add(Dense(6, activation='sigmoid'))

In [28]:
modelnya.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [29]:
modelnya.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 6,459,558
Trainable params: 6,459,558
Non-trainable params: 0
______________________________________________

In [32]:
history = modelnya.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 1117s 159ms/step - loss: 0.0622 - val_loss: 0.0461


In [33]:
history.history

{'loss': [0.06224217638373375], 'val_loss': [0.046078652143478394]}

Prediksi

In [34]:
input_text = vectorizer('I Fuckin Hate You')

In [35]:
data.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [36]:
modelnya.predict(np.array([input_text]))

array([[0.9915684 , 0.40818816, 0.95280993, 0.06685463, 0.79598355,
        0.16760081]], dtype=float32)

In [37]:
batch = test.as_numpy_iterator().next()

In [39]:
batch_x, batch_y = test.as_numpy_iterator().next()

In [43]:
(modelnya.predict(batch_x) > 0.5).astype(int)

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

Evaluate

In [31]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalCrossentropy

In [45]:
pre = Precision()
re = Recall()
acc = CategoricalCrossentropy()

In [48]:
for batch in test.as_numpy_iterator():
    x_true , y_true = batch

    yhat = modelnya.predict(x_true)

    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [49]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.7830591201782227, Recall:0.735336184501648, Accuracy:8.091609001159668


In [54]:
modelnya.save('toxicity.h5')

App

In [2]:
%pip install gradio jinja2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ellbendl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [30]:
import gradio as gr

In [31]:
modelbaru = tf.keras.models.load_model('toxicity.h5')

In [32]:
input_str = vectorizer('yo i fuckin hate you')

In [33]:
res = modelbaru.predict(np.expand_dims(input_str,0))

In [34]:
res > 0.5

array([[ True, False,  True, False,  True, False]])

In [35]:
data.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [36]:
data.columns[2:-1]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult'], dtype='object')

In [37]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = modelbaru.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(data.columns[2:-1]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [38]:
interface = gr.Interface(fn=score_comment, inputs=gr.inputs.Textbox(lines=2, placeholder='Toxic Detector by: AezersX'), outputs='text')

c:\Users\ellbendl\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\ellbendl\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Users\ellbendl\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Users\ellbendl\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [39]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://22664.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x23e0806db70>,
 'http://127.0.0.1:7860/',
 'https://22664.gradio.app')